In [2]:
import pandas as pd

In [11]:
df_1 = pd.read_csv('../data/kaggle_resume_dataset.csv')

In [4]:
df_2 = pd.read_csv('../data/website_resumes.csv')

In [9]:
df_2

,job_title,resume
0,Sr. Android App Developer,\r\nGLENN WHITE\r\nPhone - 415-***-****\r\nEma...
1,Debt Collector Customer Service,\r\nTiana Hampton\r\nDebt Collector/ Customer ...
2,Market Analyst Research Fellow,"\r\nDoug Kriegel\r\n**** ******* ******, *****..."
3,Loan Specialist Gs,\r\nPrinted Date */**/**** *:** EST USAJOBS Co...
4,Customer Service Waitress,"\r\nStephanie Brown\r\nLake Wales, FL *****\r\..."
...,...,...
11894,Customer Service,\r\nDedre Vance\r\nHutchins\r\nadpw48@r.postjo...
11895,Graphic Designer Digital Marketing,\r\nLarita Harris\r\nI am looking for a full-t...
11896,"Data Entry, Court Clerk, Administrative Assistant",\r\nArrolyn T. Wallace\r\n**** ** ***\r\nHoust...
11897,Machine Operator Cement,\r\nDAVID TONYOKWE\r\n*** ****** ****** ******...


In [28]:
print(df_1['Resume'][10])

Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details 

Data Science Assurance Associate 

Data Science Assurance Associate - Ernst & Young LLP
Skill Details 
JAVASCRIPT- Exprience - 24 months
jQuery- Exprience - 24 months
Python- Exprience - 24 monthsCompany Details 
company - Ernst & Young LLP
description - Fraud Investigations and Dispute Services   Assur